In [ ]:
pip install gnews

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.1/517.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=e4d9867dec3278729510f3fb419dc962d2435aa52f70956f0e160f2372f46582
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=1a53e252f8045ecb4256b139b11b87023011893ff473686f3f37d72ae462dcb3
  Stored in directory:

In [ ]:
from gnews import GNews
import csv

google_news = GNews()
google_news.max_results = 10
google_news.country = 'United States'
google_news.language = 'english'
google_news.start_date = (2022, 1, 1)
google_news.end_date = (2022, 12, 31)

classification_groupings = ['Earnings Report', 'Bankruptcy Speculation', 'Positive Quarterly Report', 'Negative Quarterly Report', 'Hiring Freeze', 'Layoffs Announced', 'Product Launch', 'Innovation', 'Regulatory Challenges', 'Legal Issues', 'Partnership', 'Leadership Changes' ]
valid_sources = ['nytimes.com', 'washingtonpost.com', 'wsj.com', 'npr.org', 'reuters.com', 'apnews.com', 'bbc.com', 'cnn.com', 'pbs.org', 'theatlantic.com', 'ft.com', 'bloomberg.com', 'economist.com', 'cnbc.com', 'marketwatch.com', 'businessinsider.com',  'hbr.org', 'forbes.com']
return_object = []
for classification in classification_groupings:
  keywordString = "Tesla AND " + classification
  for i in range(10):
    round_news = google_news.get_news(keywordString)
    for j in range(len(round_news)):
      title = round_news[j]['title']
      date = round_news[j]['published date']
      return_object.append((title, classification, date))

csv_filename = "classification_training_set.csv"
with open(csv_filename, mode='w', newline='') as file:
  writer = csv.writer(file)
  for row in return_object:
    writer.writerow(row)

print(f"CSV file '{csv_filename}' has been created")


KeyboardInterrupt: ignored

In [ ]:
#Preprocessing for model consumption component

csvFilePath = "/content/classification_training_set.csv"
import pandas as pd


# Read the CSV file into a pandas dataframe
column_names = ['Article Title', 'Classification', 'Date']
dataframe = pd.read_csv(csvFilePath, header=None, names = column_names)


# Display the first few rows of the dataframe to verify the data has been loaded properly
print(dataframe.head())


                                       Article Title   Classification  \
0  Elon Musk addresses Twitter takeover, possible...  Earnings Report   
1  Tesla (TSLA) Q3 2022 Earnings Call Transcript ...  Earnings Report   
2  Tesla investors to focus on demand issues in e...  Earnings Report   
3  Tesla reports $18.76 billion in revenue and re...  Earnings Report   
4  Tesla Earnings: What Happened with TSLA - Inve...  Earnings Report   

                            Date  
0  Wed, 19 Oct 2022 07:00:00 GMT  
1  Thu, 20 Oct 2022 07:00:00 GMT  
2  Wed, 19 Oct 2022 07:00:00 GMT  
3  Wed, 20 Apr 2022 07:00:00 GMT  
4  Thu, 21 Apr 2022 07:00:00 GMT  


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Assuming 'X' contains article titles and 'y' contains corresponding labels
X = dataframe['Article Title']
y = dataframe['Classification']
# Splitting data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizing text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Training a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

# Predicting on the validation set
predictions = clf.predict(X_val_tfidf)

# Evaluating the model
print(classification_report(y_val, predictions))



                           precision    recall  f1-score   support

   Bankruptcy Speculation       1.00      1.00      1.00        21
          Earnings Report       0.86      1.00      0.92        18
            Hiring Freeze       1.00      0.83      0.91        18
               Innovation       1.00      1.00      1.00        17
        Layoffs Announced       0.85      1.00      0.92        17
       Leadership Changes       1.00      1.00      1.00        19
             Legal Issues       0.86      0.83      0.84        23
Negative Quarterly Report       1.00      1.00      1.00        25
              Partnership       1.00      1.00      1.00        16
Positive Quarterly Report       1.00      0.87      0.93        23
           Product Launch       1.00      0.95      0.98        21
    Regulatory Challenges       0.79      0.86      0.83        22

                 accuracy                           0.94       240
                macro avg       0.95      0.95      0.94    

In [ ]:
#Magnitude Component of Classification System
classificationCSV = "/content/classification_training_set.csv"
teslaStockTrackerCSV = "/content/TSLA-2022-price-history.csv"

import pandas as pd

stock_data = pd.read_csv(teslaStockTrackerCSV)
stock_data['Next_Day_Price'] = stock_data['Close'].shift(-1)
column_names = ['Article Title', 'Classification', 'Date']
news_data = pd.read_csv(classificationCSV, header=None, names = column_names)

stock_data['Date'] = pd.to_datetime(stock_data['Date'])
news_data['Date'] = pd.to_datetime(news_data['Date'])
stock_data['Date'] = stock_data['Date'].dt.date
news_data['Date'] = news_data['Date'].dt.date


merged_data = pd.merge(news_data, stock_data, how='inner', left_on='Date', right_on='Date')
merged_data['Delta'] = merged_data['Next_Day_Price'] - merged_data['Close']

stock_data.to_csv('stock.csv', index=False)
magnitude_training_set = merged_data[['Classification', 'Delta']]
magnitude_training_set.to_csv('magnitude_training_set.csv', index=False)

In [ ]:
#determining magninitude coeffs now that dataset is created
data = pd.read_csv('/content/magnitude_training_set.csv')
average_values = data.groupby('Classification')['Delta'].mean().reset_index()
print(average_values)

               Classification     Delta
0      Bankruptcy Speculation -7.115666
1             Earnings Report  1.814334
2               Hiring Freeze -2.281671
3                  Innovation -2.036299
4           Layoffs Announced -6.270330
5          Leadership Changes -0.209585
6                Legal Issues  2.140831
7   Negative Quarterly Report  3.096001
8                 Partnership  1.212006
9   Positive Quarterly Report -3.198663
10             Product Launch -7.892001
11      Regulatory Challenges -7.324587
